In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df=pd.read_csv("../input/ccdata/CC GENERAL.csv")
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.fillna(method='ffill',inplace=True)
df.head()

In [ ]:

df.isnull().sum().sort_values(ascending=False).head()

In [ ]:
#we dont need customer_id to classify
df.drop(["CUST_ID"],axis=1,inplace=True)
df.head()

In [ ]:
figure = plt.figure(figsize=(16, 9))
raw_PURCHASES = figure.add_subplot(2,4,1) 
log_PURCHASES = figure.add_subplot(2,4,2) #silhouette bar chart
raw_BALANCE = figure.add_subplot(2,4,3) #silhouette bar chart
log_BALANCE = figure.add_subplot(2,4,4) #silhouette bar chart
raw_CREDIT_LIMIT = figure.add_subplot(2,4,5) #silhouette bar chart
log_CREDIT_LIMIT = figure.add_subplot(2,4,6) #silhouette bar chart
raw_MINIMUM_PAYMENTS = figure.add_subplot(2,4,7) #silhouette bar chart
log_MINIMUM_PAYMENTS = figure.add_subplot(2,4,8) #silhouette bar chart

sns.distplot(df['PURCHASES'], axlabel='Purchases Raw Data', ax=raw_PURCHASES)

purchases_log=np.log(1 + df['PURCHASES'])
sns.distplot(purchases_log, axlabel='Purchases Log Data', ax=log_PURCHASES)

sns.distplot(df['BALANCE'], axlabel='Balance Raw Data', ax=raw_BALANCE)

purchases_log=np.log(1 + df['BALANCE'])
sns.distplot(purchases_log, axlabel='Balance Log Data', ax=log_BALANCE)

sns.distplot(df['CREDIT_LIMIT'], axlabel='Credit Limit Raw Data', ax=raw_CREDIT_LIMIT)

purchases_log=np.log(1 + df['CREDIT_LIMIT'])
sns.distplot(purchases_log, axlabel='Credit Limit Log Data', ax=log_CREDIT_LIMIT)

sns.distplot(df['MINIMUM_PAYMENTS'], axlabel='Minimum Payments Raw Data', ax=raw_MINIMUM_PAYMENTS)

purchases_log=np.log(1 + df['MINIMUM_PAYMENTS'])
sns.distplot(purchases_log, axlabel='Minimum Payments Log Data', ax=log_MINIMUM_PAYMENTS)

figure.tight_layout()

In [ ]:

from sklearn.preprocessing import StandardScaler,normalize
sc=StandardScaler()
df=sc.fit_transform(df)
df=normalize(df)
df=pd.DataFrame(df)

In [ ]:
from sklearn.decomposition import PCA
pca=PCA(n_components=2)
df=pca.fit_transform(df)

In [ ]:
#elbow method to find optimal clusters
from sklearn.cluster import KMeans
wcss=[]
for i in range(1,11):
    kmeans=KMeans(n_clusters=i,init="k-means++")
    kmeans.fit(df)
    wcss.append(kmeans.inertia_)
from matplotlib import pyplot as plt
plt.figure(figsize=(8,6))
plt.plot(range(1,11),wcss)
plt.xlabel("no.of clusters")
plt.ylabel("wcss")
plt.title("Elbow Method")
plt.show()


In [ ]:

kmeans=KMeans(n_clusters=3,init="k-means++")
yhat=kmeans.fit_predict(df)

In [ ]:
plt.figure(figsize=(8,6))
plt.scatter(df[yhat==0,0],df[yhat==0,1],s=100,color="blue",label="Cluster-1")
plt.scatter(df[yhat==1,0],df[yhat==1,1],s=100,color="red",label="Cluster-2")
plt.scatter(df[yhat==2,0],df[yhat==2,1],s=100,color="green",label="Cluster-3")
plt.title("Clusters of Clients")
plt.legend()
plt.show()

In [ ]:
# Silhouette Score
s=[]
from sklearn.metrics import silhouette_score
from sklearn.cluster import AgglomerativeClustering 
for i in range(2,10):
    s.append(silhouette_score(df,AgglomerativeClustering(n_clusters=i).fit_predict(df)))
plt.figure(figsize=(8,6))
plt.bar(range(2,10), s)
plt.xlabel('Number of clusters') 
plt.ylabel('Silhouette Score') 
plt.show() 

In [ ]:
y=AgglomerativeClustering(n_clusters=2).fit_predict(df)
# Visualizing the clustering 
df=pd.DataFrame(df)
plt.figure(figsize=(8,6))
plt.scatter(df.iloc[:,0], df.iloc[:,1],  
           c = y, cmap =plt.cm.winter) 